In [ ]:
# standard imports
import numpy as np
import xarray as xr
import rioxarray
import matplotlib.pyplot as plt

# load up the saved dataset
import pickle
fp = '/Users/zachkeskinen/Documents/spicy-snow/tests/test_data/2_img_ds'
# fp = '/Users/zachkeskinen/Documents/spicy-snow/data/10_img_dB.pkl'
with open(fp, 'rb') as f:
    ds = pickle.load(f)

# add module to path so python can import it. This is the directory with 
# the __init__.py file in it to let python know this is a module.
import sys
sys.path.append('/Users/zachkeskinen/Documents/spicy-snow/spicy_snow')

# import the function to test
from processing.snow_index import calc_delta_VV, calc_delta_cross_ratio

# ds = ds.isel(time = slice(1, 3))

ds = calc_delta_cross_ratio(ds)
ds = calc_delta_VV(ds)

# import the function to test
from processing.snow_index import calc_delta_gamma, delta_gamma_clip_outlier
ds = calc_delta_gamma(ds)

In [ ]:
fcf = ds['fcf']
cr = ds['deltaCR']
vv = ds['deltaVV']
B = 0.5
deltaG_real = (1 - fcf) * cr + (fcf * B * vv)

In [ ]:
ds['deltaGamma'].isnull().sum()

In [ ]:
deltaG_real = deltaG_real.values.ravel()
deltaG_calc = ds['deltaGamma'].values.ravel()
assert np.allclose(deltaG_real[~np.isnan(deltaG_real)], deltaG_calc[~np.isnan(deltaG_calc)])

In [ ]:
ds1 = delta_gamma_clip_outlier(ds)

In [ ]:
ds1['deltaGamma'].max()

In [ ]:
ds1['deltaGamma'].min()

In [ ]:
ds1['deltaGamma'].isel(time = 1).plot()